# "Duo Net"

In [1]:
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import sys
sys.path.append('../')
from utils import DataFeed
from keras import regularizers

Using TensorFlow backend.


### Load Data

In [2]:
data_path='../preprocessing/preprocessed_data'

In [3]:
train_data, train_labels = DataFeed.Dataset.create(data_path, ['train/voxforge', 'train/youtube'], num=50000, use_premade=True)
val_data, val_labels = DataFeed.Dataset.create(data_path, ['val/youtube', 'val/voxforge'], num=-1, shuffle=True)

In [2]:
callback_stopearly = keras.callbacks.EarlyStopping(monitor='val_acc',
                                                   patience=5)

### Define Model

In [33]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=1024, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=28,
                         fmin=0.0, fmax=10000, power_melgram=1.0,
                         return_decibel_melgram=True, trainable_fb=False,
                         trainable_kernel=False))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Reshape((-1, 1)))
model.add(layers.Conv1D(16, 16, activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))

In [34]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_14 (Melspectr (None, 28, 157, 1)        1064988   
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 26, 155, 64)       640       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 13, 77, 64)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 11, 75, 64)        36928     
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 5, 37, 64)         0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 3, 35, 128)        73856     
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 1, 17, 128)        0         
__________

In [81]:
model.compile(optimizer='Rmsprop',
              metrics=['accuracy'],
              loss='categorical_crossentropy')

In [13]:
a = np.array([[1, 2], [3, 4]])

In [15]:
a

array([[1, 2],
       [3, 4]])

In [22]:
a[:, :, None].shape

(2, 2, 1)